# SQL-6

### Functions and Triggers, and Rules

---
### Setup the environment

In [ ]:
pip install ipython-sql psycopg2

In [ ]:
%load_ext sql

**Note** For this activity make sure to drop and create a new `cricdb` database. Run the following command from postgres commandline.

postgres# `drop database if exists cricdb;`

postgres# `create database cricdb;`

* Also make sure to shutdown or restart any kernels with active connection

In [2]:
%sql postgresql://postgres@localhost:5432/cricdb

---

### A simple trigger

In [ ]:
%sql drop table if exists player
%sql drop table if exists team

In [ ]:
%%sql create table player (
    player_id integer,
    first_name varchar(25),
    last_name varchar(25),
    dob date,
    team_id integer
)

In [ ]:
%%sql create table team (
    team_id integer,
    team_name varchar(50),
    team_owner varchar(50)
)

In [ ]:
%%sql create or replace function add_dummy_team() returns trigger
as $example$
    begin
        insert into team values (NEW.team_id, 'dummy', 'dummy' );
        return NEW;
    end;
$example$ language plpgsql

In [ ]:
%%sql create trigger example_trigger after insert on player
    for each row
    execute procedure add_dummy_team();

In [ ]:
%sql select * from team;

In [ ]:
%sql select * from player;

In [ ]:
%%sql insert into player 
    values (1, 'Hardik', 'Pandya', '1993-10-11', 101);

In [ ]:
%sql select * from team;

In [ ]:
%sql select * from player;

In [ ]:
%%sql insert into player 
    values (2, 'MS', 'Dhoni', '1981-07-07', 101);

---

### Deleting a trigger

In [ ]:
%sql drop trigger example_trigger on player;

In [ ]:
%%sql insert into player 
    values (3, 'Rohit', 'Sharma', '1987-04-30', 101);

Q: check contents of relations `player` and `team`

---


### Example usecase: using functions and triggers to log (or audit)

In [ ]:
%sql drop table if exists player


In [ ]:
%%sql create table player (
    player_id integer not null,
    first_name varchar(25) not null,
    last_name varchar(25) not null,
    dob date not null,
    team_id integer
)

In [ ]:
%%sql create table player_log (
    player_id integer not null,
    team_id integer,
    last_op text,
    last_update timestamp with time zone,
    primary key(player_id, last_update)
)

In [ ]:
%%sql create or replace function add_to_player_log() returns trigger
as $$
    begin
        insert into player_log
            values (OLD.player_id, old.team_id, TG_OP, now());
        return new;
    end;
$$ language plpgsql
        

In [ ]:
%%sql create trigger player_log_trigger after delete or update on player
    for each row
    execute procedure add_to_player_log();

In [ ]:
%%sql insert into player 
    values (1, 'Hardik', 'Pandya', '1993-10-11', 101);

In [ ]:
%sql select * from player_log

In [ ]:
%sql select * from player

In [ ]:
%sql update player set team_id = 102

In [ ]:
%sql delete from player where player_id = 1;

---
### Notes on Event Triggers

```
create trigger example_trigger AFTER INSERT ON table_name
    for each row ...`
```

#### Type 
* insert
* delete
* update [of column_name[,...]]

#### Granularity
* for each row
* for each statement


#### When
* Before
* After


---
Q: Try out writing different triggers based on the above aspects and find out `how many times` does the function gets executed. (when does it gets called once and when many times?)

---
Q: What are some common use cases of triggers?

---
Q: What are some advantages and disadvantages of triggers?

---

### Handling updates on views using event triggers

In [ ]:
%sql drop table if exists team

In [ ]:
%%sql create table team (
    team_id integer,
    team_name varchar(50) not null,
    team_owner varchar(50) not null,
    primary key (team_id)
)

In [ ]:
%sql drop table if exists player

In [ ]:
%%sql create table player (
    player_id integer not null,
    first_name varchar(25) not null,
    last_name varchar(25) not null,
    dob date not null,
    team_id integer,
    primary key (player_id),
    unique(player_id, dob),
    check (dob > '1980-01-31'),
    foreign key (team_id) references team (team_id)
)

In [ ]:
%%sql insert into team values
    (101, 'Mumbai Indians', 'Reliance Industry Ltd'),
    (102, 'Chennai Super Kings', 'India Cements Ltd'),
    (103, 'Delhi Capitals', 'JSW Sports')
    ;

In [ ]:
%%sql insert into player values
    (1, 'Hardik', 'Pandya', '1993-10-11', 101),
    (2, 'MS', 'Dhoni', '1981-07-07', 102),
    (3, 'Rohit', 'Sharma', '1987-04-30', 101),
    (4, 'Ruturaj', 'Gaikwad', '1997-01-31', 102);


In [ ]:
%%sql create view mi_players as
select player_id, first_name, last_name
from player join team on player.team_id = team.team_id
where team_name = 'Mumbai Indians';

In [ ]:
%sql select * from mi_players;

In [ ]:
%sql delete from mi_players where player_id = 1;

In [ ]:
%sql insert into mi_players values (5, 'Jasprit', 'Bhumra');

In [ ]:
%%sql create or replace function update_mi_players() returns trigger 
as $$
    begin 
        if (TG_OP = 'DELETE') then
            delete from player where player_id = OLD.player_id;
            if not found then return null; end if;
            return old;

        elsif (TG_OP = 'UPDATE') then
            update player set player_id = new.player_id 
                where first_name = old.first_name and last_name = old.last_name;
            if not found then return null; end if;
            return new;
        
        elsif (TG_OP = 'INSERT') then
            raise exception 'Cannnot insert into view as data of birth not known';
                
        end if;

    end;
$$ language plpgsql;


In [ ]:
%%sql create trigger check_update_on_view
    instead of insert or update or delete on mi_players
    for each row execute function update_mi_players(); 

In [ ]:
%sql delete from mi_players where player_id = 1;

In [ ]:
%sql select * from mi_players;

In [ ]:
%sql select * from player

In [ ]:
%sql insert into mi_players values (5, 'Jasprit', 'Bhumra');

In [ ]:
%sql update mi_players set player_id = 5 where first_name = 'Rohit' and last_name = 'Sharma';

---
### Using Rules


In [ ]:
%sql drop trigger if exists check_update_on_view on mi_players

Try inserting a tuple in the view again.

In [ ]:
%%sql create or replace rule view_insert as on insert to mi_players
    do instead (
        insert into player values (new.player_id, new.first_name, new.last_name, '1988-01-01', 101);
    );

In [ ]:
%sql insert into mi_players values (9, 'Jasprit', 'Bhumra');

In [ ]:
%sql select * from player

In [ ]:
%sql select * from mi_players;